In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
class Cascade_info():
    def __init__(self):
        self.view = 0
        self.click = 0
def http_cut1(ex_str):
    ex_str = ex_str.replace('http://',"")
    ex_str = ex_str.replace('https://',"")
    ex_str = ex_str.replace('www.',"")
    if len(ex_str)>0:
        if ex_str[-1]=='/':
            ex_str=ex_str[:-1]
    return ex_str
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]

In [2]:
with open('query_dict/id_querry_clean.pickle', 'rb') as file:
    id_querry_clean = pickle.load(file)

In [3]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [4]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [5]:
vals = all_groups['DocumentId'].values
qifs = all_groups['QueryId'].values

In [6]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)

In [7]:
with open("QDP_casc.pickle", 'rb') as file:
    url_casc = pickle.load(file)
with open("QHP_casc.pickle", 'rb') as file:
    host_casc = pickle.load(file)

In [8]:
casc_feats = dict()
casc_feats['QD_casc'] = []
casc_feats['QD_casc_cl'] = []
casc_feats['QD_casc_v'] = []
casc_feats['QH_casc'] = []
casc_feats['QH_casc_cl'] = []
casc_feats['QH_casc_v'] = []

In [9]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    if url_casc[Q][key1].view==0:
        casc_feats['QD_casc'].append(0.0)
    else:
        casc_feats['QD_casc'].append(url_casc[Q][key1].click/url_casc[Q][key1].view)
    casc_feats['QD_casc_cl'].append(url_casc[Q][key1].click)
    casc_feats['QD_casc_v'].append(url_casc[Q][key1].view)

100%|██████████| 606050/606050 [00:01<00:00, 544539.42it/s]


In [10]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    if host_casc[Q][key1].view==0:
        casc_feats['QH_casc'].append(0.0)
    else:
        casc_feats['QH_casc'].append(host_casc[Q][key1].click/host_casc[Q][key1].view)
    casc_feats['QH_casc_cl'].append(host_casc[Q][key1].click)
    casc_feats['QH_casc_v'].append(host_casc[Q][key1].view)

100%|██████████| 606050/606050 [00:01<00:00, 433982.81it/s]


In [11]:
with open("casc_feats_pair.pickle", 'wb') as file:
    pickle.dump(casc_feats, file)